In [1]:
%matplotlib inline
### Interactive medword testing tool ###

import preprocess as pp
import embedding
import model_validation as mv
import shared.load_config 
import json
import pylab
import os
import word2vec as w2v

pp.setup()

[nltk_data] Downloading package punkt to /usr/local/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
### script settings ###

# if you want to compute new training data from raw-data
COMPUTE_NEW_TRAIN_DATA = True
# note: if you don't compute new training data, you must provide them
#       at train_data_src for training a new model


# if you want to train a new model on training data
TRAIN_NEW_MODEL = True
# note: if you don't train a new model, you must provide it
#       at emb_model_src for the validation step

In [3]:
### define filepaths and names ###
#
# Note: when generating training data, all *.txt files in any subdirectory of 
#       raw_data_dir are considered. If you save the generated train_data in the
#       same folder (raw_data_dir), it will be used as raw_data input in a 
#       following run (undesired).

# where to get the configuration file
config_data_dir = ''
config_filename = 'configuration.json'

# where to get the raw *txt files for preprocessing
raw_data_dir = 'data/train_data/raw_data/wiki_data/positive/'

# where to store preprocessing output
train_data_src = 'dev_data/train_data/train.txt'

# where to store trained embedding model
emb_model_dir = 'dev_data/embeddings/'
emb_model_filename = 'emb_model.bin'
emb_model_src = os.path.join(emb_model_dir, emb_model_filename)

# Load configuration (don't change)
config_src = os.path.join(config_data_dir, config_filename)
config = shared.load_config.Configuration(config_src)

In [10]:
### choose your tokenizer for the prepocessing and validation ###

## NonStemmingTokenizer: 'nst'
# - no stemming, only remove punctuation marks
# - lowercase letters
config.config['tokenizer'] = 'nst'

## SimpleGermanTokenizer: 'sgt'
# - remove punctuation marks
# - stemming
# - lowercase letters
# config.config['tokenizer'] = 'sgt'

# test the chosen tokenizer
tokenizer = pp.get_tokenizer(config)
example = """!! <!--Fieber-->\n\nDas Fieber ist ein „asdf“ 'asdf' haus-ambach "asdf" ??Ein Arzt bzw. eine Ärztin ist ein medizinisch\n \n 
             ausgebildeter Heilkundiger, beschäftigt sich mit der
             Vorbeugung (Prävention), [Erkennung] (Diagnose),
             Behandlung (Therapie) [...] und Nachsorge von Krankheiten
             und Leiden bzw. gesundheitlichen Beeinträchtigungen,
             und stellt sich damit in den Dienst der Gesundheit."""
             
tk_example = tokenizer.tokenize(str(example))

print(tk_example)

['<', 'fieber', '>', 'das', 'fieber', 'ist', 'ein', 'asdf', 'asdf', 'haus-ambach', 'asdf', 'ein', 'arzt', 'bzw', 'eine', 'aerztin', 'ist', 'ein', 'medizinisch', 'ausgebildeter', 'heilkundiger', 'beschaeftigt', 'sich', 'mit', 'der', 'vorbeugung', 'praevention', 'erkennung', 'diagnose', 'behandlung', 'therapie', 'und', 'nachsorge', 'von', 'krankheiten', 'und', 'leiden', 'bzw', 'gesundheitlichen', 'beeintraechtigungen', 'und', 'stellt', 'sich', 'damit', 'in', 'den', 'dienst', 'der', 'gesundheit']


In [ ]:
### compute new training data from raw *txt files ###

# compute new train data if needed
if (COMPUTE_NEW_TRAIN_DATA):
    pp.create_train_data(train_data_src, raw_data_dir, config)

In [5]:
### train a new model on your training data ###
# note: the used configuration file is saved together with the model
#       in the emb_model_dir
#

# dimension of embedding vectors
config.config['embedding_vector_dim'] = 400

# minimum number of appearence fore a token to be considered in the 
# model (you may want to ignore rare tokens)
config.config['min_token_appearance'] = 1


# train embeddings using word2vec
if (TRAIN_NEW_MODEL):
    embedding.make_emb_from_file(train_data_src, emb_model_dir, emb_model_filename, config)

Start training the model.
Starting training using file dev_data/train_data/train.txt
Vocab size: 134927
Words in train file: 1268039
Alpha: 0.000089  Progress: 100.08%  Words/thread/sec: 186.08k  Training finsihed. 
Model saved at: dev_data/embeddings/emb_model.bin


In [6]:
### Validation of the embedding model ###

# Choose test files from your validation folder
config.config['doesntfit_file'] = 'german_doesntfit1.txt'
config.config['synonyms_file'] = 'german_synonyms3.txt'


model = mv.validate_model(emb_model_src, config)

Start validation. Loading model. 

Validating 'doesntfit' with file data/validation_data/german_doesntfit1.txt

*** Doesn't fit ***
Doesn't fit correct:  43.9% (25/57)
Doesn't fit coverage: 51.8% (57/110) 

Validating 'synonyms' with file data/validation_data/german_synonyms3.txt
Synonyms: 42175 pairs in input. 1649 pairs in model-vocabulary.

*** Cosine-Similarity ***
Synonyms avg-cos-similarity (SACS): 0.959299000128 
Random avg-cos-similarity (RACS): 0.882233701681 
Ratio SACS/RACS: 1.08735247622

*** Synonym Recognition ***
Synonyms correct:  0.3% (11/3298), checked 40 closest embedding-vectors.checked per word.
Synonyms coverage: 3.9% (3298/84350)



In [9]:
### Load model separately if desired ###
# note: also loaded in mv.validate_model() ###
model = w2v.load(emb_model_src)

In [ ]:
### Get model information ###
print("vocab_size:", model.vocab.size)  # note that the token '</s>' is 
                                        # always added to the model
    
print("vectors_shape:", model.vectors.shape)

In [ ]:
### Play with model ###

# define a word from which you like to see it's n-nearest neighbours 
# in the embedding space
word = 'arzt'
n_nearest_neighbours = 10

# create list of neighbours
indexes, metrics = model.cosine(word, n_nearest_neighbours)
model.generate_response(indexes, metrics).tolist()

In [ ]:
# Get a vector of a word
vec1 = model['arzt'] 
# or
vec2 = model.get_vector('arzt')

print(vec1[:10])

In [ ]:
### Visualization ###
# From a list of words, plot all words and it's neighbours 

# define the words you would like to visualize
word_list = ['arzt', 'haus', 'natur']

# define the number of closest neighbors to display per word
n_nearest_neighbours = 5

# change figure size if desired
width = 10.0
height = 8.0
pylab.rcParams['figure.figsize'] = (width, height)

# plotting
mv.visualize_words(model, word_list, n_nearest_neighbours)